In [1]:
import pandas as pd
import yfinance as yf
import yahoofinancials as YahooFinancials
from delta import *
import pyspark
import os
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType,BooleanType,DateType

In [2]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/05/14 15:08:39 WARN Utils: Your hostname, MacBook-Pro-de-Renato.local resolves to a loopback address: 127.0.0.1; using 10.0.0.102 instead (on interface en0)
22/05/14 15:08:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/renabouj/Codes/tendenciasSI/env/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/renabouj/.ivy2/cache
The jars for the packages stored in: /Users/renabouj/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-00ae8707-5fa7-48df-b80d-c1bc3fb12501;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 163ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnld

In [3]:
dataf = spark.read.option("header",True).csv("data/2msales.csv")
dataf.show()

+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|    Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|           Australia|         Meat|       Online|             C|  4/4/2011|451691138| 5/23/2011|      4300|    421.89|   364.69|   1814127.00|1568167.00|   245960.00|
|                Asia|          Tajikistan|Personal Care|       Online|             L| 7/12/2018|144177377|  8/1/2018|      4145|     81.73|    56.67|    338770.85| 234897.15|   103873.70|
|  Sub-Saharan Africa|          Mozambique|    Cosmetic

In [4]:
dataf = dataf.withColumnRenamed("Item Type", "item_type")
dataf = dataf.withColumnRenamed("Sales Channel", "sales_channel")
dataf = dataf.withColumnRenamed("Order Priority", "order_priority")
dataf = dataf.withColumnRenamed("Order Date", "order_date")
dataf = dataf.withColumnRenamed("Order ID", "order_id")
dataf = dataf.withColumnRenamed("Ship Date", "ship_date")
dataf = dataf.withColumnRenamed("Ship Date", "ship_date")
dataf = dataf.withColumnRenamed("Units Sold", "units_sold")
dataf = dataf.withColumnRenamed("Unit Price", "unit_price")
dataf = dataf.withColumnRenamed("Unit Cost", "unit_cost")
dataf = dataf.withColumnRenamed("Total Revenue", "total_revenue")
dataf = dataf.withColumnRenamed("Total Cost", "total_cost")
dataf = dataf.withColumnRenamed("Total Profit", "total_profit")

In [6]:
dataf = dataf.withColumn('data_do_pedido', col('order_date').cast('timestamp'))
dataf = dataf.withColumn("data_de_envio", col('ship_date').cast('timestamp'))

#dataf = dataf.with

In [7]:
dataf.show()

+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+--------------+-------------+
|              Region|             Country|    item_type|sales_channel|order_priority|order_date| order_id| ship_date|units_sold|unit_price|unit_cost|total_revenue|total_cost|total_profit|data_do_pedido|data_de_envio|
+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+--------------+-------------+
|Australia and Oce...|           Australia|         Meat|       Online|             C|  4/4/2011|451691138| 5/23/2011|      4300|    421.89|   364.69|   1814127.00|1568167.00|   245960.00|          null|         null|
|                Asia|          Tajikistan|Personal Care|       Online|             L| 7/12/2018|144177377|  8/1/2018|      4145

In [8]:
dataf = dataf.withColumn("units_sold", dataf.units_sold.cast('bigint'))
dataf = dataf.withColumn("unit_price", dataf.unit_price.cast('bigint'))
dataf = dataf.withColumn("unit_cost", dataf.unit_cost.cast('bigint'))
dataf = dataf.withColumn("total_revenue", dataf.total_revenue.cast('bigint'))
dataf = dataf.withColumn("total_cost", dataf.total_cost.cast('bigint'))
dataf = dataf.withColumn("total_profit", dataf.total_profit.cast('bigint'))

In [9]:
dataf.dtypes

[('Region', 'string'),
 ('Country', 'string'),
 ('item_type', 'string'),
 ('sales_channel', 'string'),
 ('order_priority', 'string'),
 ('order_date', 'string'),
 ('order_id', 'string'),
 ('ship_date', 'string'),
 ('units_sold', 'bigint'),
 ('unit_price', 'bigint'),
 ('unit_cost', 'bigint'),
 ('total_revenue', 'bigint'),
 ('total_cost', 'bigint'),
 ('total_profit', 'bigint'),
 ('data_do_pedido', 'timestamp'),
 ('data_de_envio', 'timestamp')]

In [26]:
dataf.describe(['id']).show()

+-------+--------------------+
|summary|                  id|
+-------+--------------------+
|  count|             2000000|
|   mean|2.955984681875748E10|
| stddev|1.945317355514702...|
|    min|                   0|
|    max|         60129762764|
+-------+--------------------+



In [25]:
dataf.show()

+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+---+
|              Region|             Country|    item_type|sales_channel|order_priority|order_date| order_id| ship_date|units_sold|unit_price|unit_cost|total_revenue|total_cost|total_profit| id|
+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+---+
|Australia and Oce...|           Australia|         Meat|       Online|             C|  4/4/2011|451691138| 5/23/2011|      4300|       421|      364|      1814127|   1568167|      245960|  0|
|                Asia|          Tajikistan|Personal Care|       Online|             L| 7/12/2018|144177377|  8/1/2018|      4145|        81|       56|       338770|    234897|      103873|  1|
|  Sub-Saharan Africa|          Moz

In [10]:
%timeit
dataf.write.parquet("/tmp/output/arquivodeteste.parquet")

22/05/14 15:09:29 WARN MemoryManager: Total allocation exceeds 95,00% (906.992.014 bytes) of heap memory
Scaling row group sizes to 96,54% for 7 writers
22/05/14 15:09:29 WARN MemoryManager: Total allocation exceeds 95,00% (906.992.014 bytes) of heap memory
Scaling row group sizes to 84,47% for 8 writers
22/05/14 15:09:35 WARN MemoryManager: Total allocation exceeds 95,00% (906.992.014 bytes) of heap memory
Scaling row group sizes to 96,54% for 7 writers


In [11]:
%timeit
dataf.write.format("delta").save("/tmp/output/arquivodetestedelta")

22/05/14 15:09:36 WARN MemoryManager: Total allocation exceeds 95,00% (906.992.014 bytes) of heap memory
Scaling row group sizes to 96,54% for 7 writers
22/05/14 15:09:36 WARN MemoryManager: Total allocation exceeds 95,00% (906.992.014 bytes) of heap memory
Scaling row group sizes to 84,47% for 8 writers
22/05/14 15:09:41 WARN MemoryManager: Total allocation exceeds 95,00% (906.992.014 bytes) of heap memory
Scaling row group sizes to 96,54% for 7 writers


In [12]:
dataf.write.format("csv").save("/tmp/output/csvdeteste")

In [96]:
%timeit
parDF = spark.read.parquet("/tmp/output/arquivodeteste.parquet")

In [97]:
%%timeit
parDF.show()

+--------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|              Region|   Country|      item_type|sales_channel|order_priority|order_date| order_id| ship_date|units_sold|unit_price|unit_cost|total_revenue|total_cost|total_profit|         id|
+--------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|  Sub-Saharan Africa|   Nigeria|      Baby Food|      Offline|             C|12/23/2017|977884328|  1/8/2018|      6558|       255|      159|      1674126|   1045476|      628649|25769803776|
|Middle East and N...|     Syria|      Beverages|      Offline|             H| 9/21/2016|670741355| 11/6/2016|      6127|        47|       31|       290726|    194777|       95948|25769803777|
|Central America a...|      Cuba|  

In [29]:
%timeit
df = spark.read.format("delta").load("/tmp/output/arquivodetestedelta")
df.show()

+--------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|              Region|   Country|      item_type|sales_channel|order_priority|order_date| order_id| ship_date|units_sold|unit_price|unit_cost|total_revenue|total_cost|total_profit|         id|
+--------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|  Sub-Saharan Africa|   Nigeria|      Baby Food|      Offline|             C|12/23/2017|977884328|  1/8/2018|      6558|       255|      159|      1674126|   1045476|      628649|25769803776|
|Middle East and N...|     Syria|      Beverages|      Offline|             H| 9/21/2016|670741355| 11/6/2016|      6127|        47|       31|       290726|    194777|       95948|25769803777|
|Central America a...|      Cuba|  

In [19]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")

In [20]:
teste = DeltaTable.forPath(spark, "/tmp/output/arquivodetestedelta")

In [36]:
teste.update(
  condition = expr("id == 25769803776"),
  set = { "units_sold": expr("units_sold + 4000") })

In [38]:
teste.update(
  condition = expr("id == 25769803776"),
  set = { "unit_price": expr("unit_price + 5") })

In [39]:
opa = spark.read.format("delta").load("/tmp/output/arquivodetestedelta")
opa.where(opa.id == 25769803776).show()

+------------------+-------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+-----------+
|            Region|Country|item_type|sales_channel|order_priority|order_date| order_id|ship_date|units_sold|unit_price|unit_cost|total_revenue|total_cost|total_profit|         id|
+------------------+-------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+-----------+
|Sub-Saharan Africa|Nigeria|Baby Food|      Offline|             C|12/23/2017|977884328| 1/8/2018|     10000|       260|      159|      1674126|   1045476|      628649|25769803776|
+------------------+-------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+-----------+



In [22]:
newData = spark.range(0, 20)
deltaTable.alias("oldData").merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



In [23]:
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "/tmp/checkpoint").start("/tmp/delta-table")

22/05/14 10:31:31 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [24]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows



In [25]:
stream.stop()

22/05/14 10:33:18 ERROR ShuffleBlockFetcherIterator: Error occurred while fetching local blocks, null
22/05/14 10:33:18 WARN BlockManager: Block rdd_2779_46 could not be removed as it was not found on disk or in memory
22/05/14 10:33:18 WARN BlockManager: Putting block rdd_2779_40 failed due to exception org.apache.spark.TaskKilledException.
22/05/14 10:33:18 WARN BlockManager: Block rdd_2779_40 could not be removed as it was not found on disk or in memory
22/05/14 10:33:18 WARN BlockManager: Putting block rdd_2779_45 failed due to exception org.apache.spark.TaskKilledException.
22/05/14 10:33:18 WARN BlockManager: Block rdd_2779_45 could not be removed as it was not found on disk or in memory
22/05/14 10:33:18 WARN BlockManager: Putting block rdd_2779_39 failed due to exception org.apache.spark.TaskKilledException.
22/05/14 10:33:18 WARN BlockManager: Block rdd_2779_39 could not be removed as it was not found on disk or in memory
22/05/14 10:33:18 WARN TaskSetManager: Lost task 45.0 i